In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import dask.dataframe as dd
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from group_lasso import GroupLasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_errorp

import gc
import sqlite3
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'seaborn'

In [3]:
pd.read_csv("c:\Code_data\Xwines\Xwines.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Code_data\\Xwines\\Xwines.csv'

In [ ]:
%load_ext sql
%sql sqlite://

In [ ]:
# Connect to SQLite database (in memory)
conn = sqlite3.connect(':memory:')  # Use ':memory:' for in-memory database

# Insert DataFrame into a table named 'my_table' in SQLite database
df.to_sql('my_table', conn, index=False, if_exists='replace')


In [ ]:
query = "SELECT rating FROM my_table;"
result_df = pd.read_sql_query(query, conn)

In [ ]:
result_df

In [ ]:
query = "SELECT WineName FROM my_table;"
X = pd.read_sql_query(query, conn)

In [ ]:
result = %sql SELECT rating FROM my_table

In [ ]:
db_path = '/content/SQL.db'
conn = sqlite3.connect(db_path)

In [ ]:
engine = create_engine('sqlite:///xwines.db', echo=False)

# Write data to SQLite, replacing if the table already exists
df.to_sql('xwines_table', con=engine, if_exists='replace', index=False)

# Now you can query the database
with engine.connect() as con:
    result = con.execute("SELECT  Rating, Type,WineName,Acidity,Country FROM xwines_table  LIMIT 5 ")
    for row in result:
        print(row)

In [ ]:

df_reg = result[['Type', "Acidity" ,"WineName", "Country"]]

# 将分类变量转换为虚拟/哑变量
df_dum = pd.get_dummies(df_reg,drop_first=True)


X = df_dum
y = result['Rating']

# 初始化线性回归模型
model = LinearRegression()

# 拟合模型
model.fit(X, y)

# 打印系数和截距
model.summary()


In [ ]:
plt.figure(figsize=(4, 3))
plt.hist(df1["Rating"], bins=10, color='skyblue', edgecolor='black')

In [ ]:
df2 = pd.read_csv("C:\Code_data\Xwines(1)\XWines_Full_100K_wines.csv")
# df2.head(3)
# df2.info()

In [ ]:
print(len(df2["Grapes"].unique()))
print(len(df2["Harmonize"].unique()))
print(len(df2["Elaborate"].unique()))
print(len(df2["Country"].unique()))
print(len(df2["WineName"].unique()))
print(len(df2["RegionName"].unique()))
print(len(df2["Body"].unique()))
print(len(df2["Type"].unique()))
print(len(df2["Acidity"].unique()))

In [ ]:
plt.figure(figsize=(12, 36))  # 調整整體圖表的大小，高度增加以容納垂直排列的子圖

plt.subplot(6, 1, 1)  # 6行1列的第1張圖
sns.distplot(df2['ABV'])

plt.subplot(6, 1, 2)  # 6行1列的第2張圖
sns.histplot(df2['Body'], bins=len(df2['Body'].unique()))
plt.xticks(rotation=45)

plt.subplot(6, 1, 3)  # 6行1列的第3張圖
sns.histplot(df2['Acidity'], bins=len(df2['Acidity'].unique()))

plt.subplot(6, 1, 4)  # 6行1列的第4張圖
sns.histplot(df2['Type'], bins=len(df2['Type'].unique()))

plt.subplot(6, 1, 5)  # 6行1列的第5張圖
sns.histplot(df2['Country'], bins=len(df2['Country'].unique()))
plt.xticks(rotation=90)  # 旋轉標籤以避免重疊

plt.subplot(6, 1, 6)  # 6行1列的第6張圖
sns.histplot(df2['Elaborate'], bins=len(df2['Elaborate'].unique()))
plt.xticks(rotation=45)

plt.tight_layout()  # 調整子圖之間的間距
plt.show()


In [ ]:
df = pd.merge(df1, df2, on='WineID', how='inner')
df.head(3)

In [ ]:
df.to_csv('Xwines.csv', index=False)
import os
os.listdir()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 假设df是你的DataFrame
# 如果 "Body" 和 "Acidity" 是分类变量，需要先将其编码成数值型
le = LabelEncoder()
df['Body'] = le.fit_transform(df['Body'])
df['Acidity'] = le.fit_transform(df['Acidity'])

# 确保 Rating 是目标变量，并且是连续变量
X = df[['Body', 'Acidity']]  # 注意此处的写法，应该是双重方括号
y = df['Rating']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化随机森林回归器，可以指定 n_estimators 和其他参数
reg = RandomForestRegressor(n_estimators=100)  # 也可以修改其他参数
reg.fit(X_train, y_train)

# 使用reg.predict(X_test)来进行预测


In [ ]:
# 假设'Very full-bodied'和'High'分别是你想预测的'Body'和'Acidity'的值
# 假设le_body和le_acidity是对应于'Body'和'Acidity'的LabelEncoder实例
x1 = le.transform(["Very full-bodied"])  # 这里的le应该是特定于'Body'的编码器
x2 = le.transform(["High"])            # 同上，这里的le应该是特定于'Acidity'的编码器

# 将输入值转换为模型预期的格式
X_new = np.array([[x1, x2]])

# 使用模型进行预测
prediction = reg.predict(X_new)

print(prediction)
